<h1><b>RESULTS BASED ON DOCUMENTATION RESULTS</b> <br>
<i>(ALTHOUGH, I WAS UNABLE TO REPLICATE THE RDA OF 100%)<i>:</h1>

   "The data was used with many others for comparing various
   classifiers.   
   The classes are separable, though only RDA has achieved 100% correct classification.  
   (RDA : 100%,   
   QDA 99.4%,   
   LDA 98.9%,   
   1NN 96.1% (z-transformed data))  
   (All results using the leave-one-out technique)"  

In [ ]:
!pip install -q pandas scikit-learn
import pandas as pd
from sklearn.model_selection import LeaveOneOut
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis, LinearDiscriminantAnalysis
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
import numpy as np

In [ ]:
class RegularizedDiscriminantAnalysis:
    def __init__(self, alpha=0.5):
        self.alpha = alpha
        self.means_ = None
        self.covariance_ = None
        self.priors_ = None

    def fit(self, X, y):
        self.classes_ = np.unique(y)
        n_classes = len(self.classes_)
        n_features = X.shape[1]

        self.means_ = np.zeros((n_classes, n_features))
        self.covariance_ = np.zeros((n_features, n_features))
        self.priors_ = np.zeros(n_classes)

        for idx, label in enumerate(self.classes_):
            X_class = X[y == label]
            self.means_[idx, :] = np.mean(X_class, axis=0)
            self.covariance_ += np.cov(X_class, rowvar=False) * (len(X_class) - 1)
            self.priors_[idx] = len(X_class) / len(y)

        self.covariance_ /= (len(y) - n_classes)
        pooled_covariance = np.cov(X, rowvar=False) * (len(y) - 1) / len(y)
        self.covariance_ = (1 - self.alpha) * pooled_covariance + self.alpha * self.covariance_

    def predict(self, X):
        n_samples = X.shape[0]
        n_classes = len(self.classes_)
        log_likelihood = np.zeros((n_samples, n_classes))

        for idx, label in enumerate(self.classes_):
            mean = self.means_[idx]
            covariance_inv = np.linalg.inv(self.covariance_)
            diff = X - mean
            log_likelihood[:, idx] = -0.5 * np.sum(diff @ covariance_inv * diff, axis=1) \
                                     + np.log(self.priors_[idx])

        return self.classes_[np.argmax(log_likelihood, axis=1)]


In [ ]:
# Load the dataset
data_path = 'https://raw.githubusercontent.com/TranDenyDFW/datasets/main/Classification/wine%20(archive.ics.uci.edu)/wine.data'
column_names = [
    'Class', 'Alcohol', 'Malic acid', 'Ash', 'Alcalinity of ash',
    'Magnesium', 'Total phenols', 'Flavanoids', 'Nonflavanoid phenols',
    'Proanthocyanins', 'Color intensity', 'Hue', 'OD280/OD315 of diluted wines', 'Proline'
]
wine_data = pd.read_csv(data_path, header=None, names=column_names)

# Separate features and target variable
X = wine_data.drop('Class', axis=1)
y = wine_data['Class']

# Standardize the features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Leave-One-Out Cross-Validation
loo = LeaveOneOut()

def evaluate_model(model, X, y):
    y_true, y_pred = [], []
    for train_index, test_index in loo.split(X):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
        model.fit(X_train, y_train)
        y_true.append(y_test[0])
        y_pred.append(model.predict(X_test)[0])
    return accuracy_score(y_true, y_pred)

# Initialize models
qda = QuadraticDiscriminantAnalysis()
lda = LinearDiscriminantAnalysis()
knn = KNeighborsClassifier(n_neighbors=1)
rda = RegularizedDiscriminantAnalysis(alpha=0.5)

# Evaluate models
qda_accuracy = evaluate_model(qda, X_scaled, y.values)
lda_accuracy = evaluate_model(lda, X_scaled, y.values)
knn_accuracy = evaluate_model(knn, X_scaled, y.values)
rda_accuracy = evaluate_model(rda, X_scaled, y.values)


In [ ]:
accuracy_data = {
    'Model': ['QDA', 'RDA', 'LDA', 'k-NN (1NN)'],
    'Accuracy': [qda_accuracy, rda_accuracy, lda_accuracy, knn_accuracy]
}

pd.DataFrame(accuracy_data)

,Model,Accuracy
0,QDA,0.994382
1,RDA,0.988764
2,LDA,0.988764
3,k-NN (1NN),0.955056
